In [ ]:
#loading library
import numpy as np
import scipy as sp
import pandas as pd
from sklearn.cluster import KMeans

In [ ]:
#loading data
data = pd.read_table('EMGaussian.data', header = None, sep='\s+', names = ('x1', 'x2'))
test = pd.read_table('EMGaussian.test', header = None, sep='\s+', names = ('x1', 'x2'))
X = np.array(data)

In [ ]:
X

In [ ]:
#initialization
K = 4
[N,d] = data.shape #dimension of x
kmean_res = KMeans(init = 'random', n_clusters=4).fit(data)
lmu = kmean_res.cluster_centers_
lsigma = [np.eye(2,2) for k in range(K)]
lpi = [1/K for k in range(K)]

In [ ]:
lmu

In [ ]:
def compute_loglikelihood(X, Q, lpi, lmu, lsigma):
    (N,d)=X.shape
    lglklhd=0
    for i in range(N):
        for k in range(K):
            lglklhd += Q[i,k]*np.log(sp.stats.multivariate_normal.pdf(X[i,:],lmu[k,:], lsigma[k])) + Q[i,k]*np.log(lpi[k])
    return lglklhd

In [ ]:
#lpi is 1d-array; lmu, X are 2d-arrays; lsig is a list

Q = np.zeros((N,K))
condition = False
count = 0
tol=0.001

lglklhd=compute_loglikelihood(X,Q,lpi,lmu,lsigma)

while(condition==False & count < 30):
    
    count += 1
#E-step
    for i in range(N):
        denom=0
        for k in range(K):
            denom=denom+lpi[k]*sp.stats.multivariate_normal.pdf(X[i,:],lmu[k,:], lsigma[k])
        for k in range(K):
            Q[i,k]=lpi[k]*sp.stats.multivariate_normal.pdf(X[i,:],lmu[k,:], lsigma[k])/denom

#M-step
    for k in range(K):
        #compute mu
        denom_mu=np.zeros(2)
        for i in range(N):
            denom_mu += X[i,:]*Q[i,k]
        lmu[k,:]= denom_mu/np.sum(Q[:,k])
        #compute sigma
        denom_sigma=np.matlib.zeros((2,2))
        for i in range(N):
            A=np.asmatrix(X[i,:]-lmu[k])
            denom_sigma += A.T.dot(A)
        lsigma[k]=np.asarray(denom_sigma/np.sum(Q[:,k]))
        #compute pi
        lpi[k]=np.sum(Q[:,k])/np.sum(Q)
        
        lg_past=lglklhd
        lglklhd=compute_loglikelihood(X,Q,lpi,lmu,lsigma)
        if (np.abs(lglklhd-lg_past<tol)):
            condition=True
        
    

In [ ]:
lmu

In [ ]:
count

In [ ]:
X[:,0]

In [ ]:
#now let's plot 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from plotly.graph_objs import Scatter, Figure, Layout

trace = Scatter(
    x = X[:,0],
    y = X[:,1],
    mode = 'markers'
)

centroids = Scatter(
    x = lmu[:,0]
    y = lmu[:,1]
)

data = [trace]

# Plot and embed in ipython notebook!
iplot(data, filename='basic-scatter')

In [ ]:
k=1
np.dot(np.transpose((X - lmu[k])),(X - lmu[k]))

In [ ]:
lmu[1]

In [ ]:
mu0
sigma0

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from plotly.graph_objs import Scatter, Figure, Layout

iplot([Scatter(x=[1, 2, 3], y=[3, 1, 6])])